In [1]:
import itertools
import torch
import pickle
import numpy as np
import ZH_Nakamura
from Class_R import R
torch.set_default_dtype(torch.float64)

Welcome to JupyROOT 6.24/06


In [11]:
def make_weight_ratio(r, weights, base_points):
    output=0
    nCoefficients=len(r.coefficient_names)
    row,column=np.triu_indices(nCoefficients+1)
    for i_comb, combination in enumerate(r.combination_list):
        if row[i_comb]==column[i_comb]:
            factor=1
        else:
            factor=1
        output += torch.outer(torch.from_numpy(weights[combination]/weights[()]), factor*base_points[:,row[i_comb]]*base_points[:,column[i_comb]])
    return output

In [12]:
def f_loss(r,features,base_points,weights):
        fhat = 1./(1. + r.predict_r_hat(features,base_points))
        weight_ratios=make_weight_ratio(r, weights, base_points)
        loss = (torch.from_numpy(weights[()])*torch.transpose(weight_ratios*fhat**2 + (1-fhat)**2,0,1)).sum()
        return loss

In [13]:
class R:
    def __init__(self,nfeatures,coefficient_names):
        self.nfeatures         = nfeatures
        self.coefficient_names = coefficient_names
        self.combination_list=list(itertools.chain.from_iterable(itertools.combinations_with_replacement(self.coefficient_names, i) for i in np.arange(0,3)))
        self.n_hat = {combination: self.make_NN() for combination in self.combination_list}
        
    def make_NN(self, hidden_layers  = [32, 32, 32, 32]):
        model_nn = [torch.nn.BatchNorm1d(self.nfeatures), torch.nn.ReLU(), torch.nn.Linear(self.nfeatures, hidden_layers[0])]
        for i_layer, layer in enumerate(hidden_layers):
            model_nn.append(torch.nn.Linear(hidden_layers[i_layer], hidden_layers[i_layer+1] if i_layer+1<len(hidden_layers) else 1))
            if i_layer+1<len(hidden_layers):
                model_nn.append( torch.nn.ReLU() )
        return torch.nn.Sequential(*model_nn)

    def evaluate_NN(self, features):
        '''Evaluate Neural Network: The zeroth dimension of features is the number of data points and and the first dimension
        is the number of features(variables)
        '''
        noutputs=len(self.combination_list)
        ndatapoints=features.shape[0]
        
        output=torch.zeros((noutputs,ndatapoints))
        for i in range(noutputs):
            #self.n_hat[self.combination_list[i]].eval()
            x=self.n_hat[self.combination_list[i]](features)
            if i==0:
                output[i,:]=1
            else:
                output[i,:]=torch.flatten(x)            
        return output
        
    
    def predict_r_hat(self, features, theta):
        '''Evaluate positive xsec ratio for given theta and 
        '''
        ndatapoints=features.shape[0]
        output_NN = self.evaluate_NN(features)
        n_terms=len(self.coefficient_names)
        row,column=np.triu_indices(n_terms+1)
        Omega=torch.zeros((n_terms+1,n_terms+1,ndatapoints))
        for i in range(0, len(row)):
            Omega[row[i]][column[i]][:]=output_NN[i,:]
        Omega_swapped=torch.swapaxes(Omega,1,2)
        Omega_swapped=torch.swapaxes(Omega_swapped,0,1)
        
        out=torch.matmul(Omega_swapped,torch.transpose(theta,0,1))
        return torch.linalg.norm(out, 2, 1)
    
    def save(self,fileName):
        outfile = open(fileName,'wb')
        pickle.dump(self, outfile)
        outfile.close()
        
    @classmethod
    def load(self, fileName):
        infile = open(fileName,'rb')
        print(fileName)
        new_dict = pickle.load(infile)
        infile.close()
        return new_dict

In [14]:
n_features=6
#coefficients=['cHQ3', 'cHW', 'cHWtil']
#base_points = np.asarray([np.array([1, value1, value2, value3]) for value1 in [-1.5, -.8, .2, 0., .2, .8, 1.5]  for value2 in [-1.5, -.8, .2, 0, .2, .8, 1.5] for value3 in [-1.5, -.8, .2, 0, .2, .8, 1.5]])
#coefficients=['cHW', 'cHWtil']
#base_points = np.asarray([np.array([1, value1, value2]) for value1 in [-1.5, -.8, .2, 0., .2, .8, 1.5]  for value2 in [-1.5, -.8, .2, 0, .2, .8, 1.5]])
coefficients=['cHQ3']
base_points = np.asarray([np.array([1, value1]) for value1 in [-1.5, -.8, .2, 0., .2, .8, 1.5]])
print(base_points.shape)
r_NN = R(n_features, coefficients)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

(7, 2)


In [15]:
plot_directory="v1_with_Biasing"

nEvents=30000
learning_rate = 1e-3
device        = 'cuda' if torch.cuda.is_available() else 'cpu'
n_epoch       = 10000
plot_every    = 100

In [16]:
# training data
import ZH_Nakamura 
ZH_Nakamura.feature_names = ZH_Nakamura.feature_names[0:6] # restrict features
features   = ZH_Nakamura.getEvents(nEvents)[:,0:6]
feature_names  = ZH_Nakamura.feature_names
plot_options   = ZH_Nakamura.plot_options
plot_vars      = ZH_Nakamura.feature_names

mask       = (features[:,feature_names.index('pT')]<900) & (features[:,feature_names.index('sqrt_s_hat')]<1800) 
features = features[mask]

n_features = len(features[0])
weights    = ZH_Nakamura.getWeights(features, ZH_Nakamura.make_eft())


pT=features[:,feature_names.index('pT')]
features=torch.from_numpy(features)
#for key,value in weights.items():
 #   value*=bias_factor
  #  weights[key]=value

Requested 30000 events. Simulated 30000 events and 30000 survive pT_min cut of 0.


In [17]:
all_params=[]
for comb in r_NN.combination_list:
        all_params+=r_NN.n_hat[comb].parameters()

In [18]:
#optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)
optimizer = torch.optim.Adam(list(all_params), lr=learning_rate)
losses = []
for comb in r_NN.combination_list:
    r_NN.n_hat[comb].train()

In [ ]:
for epoch in range(n_epoch):
    loss = f_loss(r_NN,features,torch.from_numpy(base_points),weights)
    
    if epoch % 100 == 99:
        print("epoch", epoch, "loss",  loss.item())
        print("Loss:")
        print(loss)
        
    losses.append(loss.item())
    optimizer.zero_grad()    
    loss.backward()
    optimizer.step()

epoch 99 loss 0.7530355253884699
Loss:
tensor(0.7530, grad_fn=<SumBackward0>)
epoch 199 loss 0.7383465722702608
Loss:
tensor(0.7383, grad_fn=<SumBackward0>)
epoch 299 loss 0.7178787652048446
Loss:
tensor(0.7179, grad_fn=<SumBackward0>)
epoch 399 loss 0.7042320116241736
Loss:
tensor(0.7042, grad_fn=<SumBackward0>)
epoch 499 loss 0.6863276835743011
Loss:
tensor(0.6863, grad_fn=<SumBackward0>)
epoch 599 loss 0.638567328440903
Loss:
tensor(0.6386, grad_fn=<SumBackward0>)
epoch 699 loss 0.6350074084181669
Loss:
tensor(0.6350, grad_fn=<SumBackward0>)
epoch 799 loss 0.634416085520451
Loss:
tensor(0.6344, grad_fn=<SumBackward0>)
epoch 899 loss 0.6338122377437175
Loss:
tensor(0.6338, grad_fn=<SumBackward0>)
epoch 999 loss 0.6331632260006564
Loss:
tensor(0.6332, grad_fn=<SumBackward0>)
epoch 1099 loss 0.6327114354714574
Loss:
tensor(0.6327, grad_fn=<SumBackward0>)
epoch 1199 loss 0.6323133815768558
Loss:
tensor(0.6323, grad_fn=<SumBackward0>)
epoch 1299 loss 0.6319494458852626
Loss:
tensor(0.631

In [ ]:
weights[()]